In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# library regular expresion untuk membersihkan text dari data
import re
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import Counter
import math
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
gizi_df= pd.read_csv('https://drive.google.com/uc?id=138anPqj-DF8Af_1HS6FRwkubr34SsK6n')
gizi_df.head()

,Nama,JK,Usia Saat Ukur,Tgl Lahir,Nama Ortu,Desa/Kel,RT,RW,Berat,Tinggi,LiLA,ZS BB/U,ZS TB/U,ZS BB/TB,STATUS GIZI
0,HANNA BENISA,P,1 Tahun - 5 Bulan - 0 Hari,2/9/2024,Asmaul husnah,MOROBAKUNG,4,2.0,8.80,78.0,15.0,-1.07,-0.57,-1.11,Gizi Baik
1,MOH.ARSYA AL FARISQI,L,0 Tahun - 4 Bulan - 4 Hari,3/5/2025,Nindy,MOROBAKUNG,1,1.0,6.60,62.0,0.0,-0.62,-1.05,0.14,Gizi Baik
2,MUHAMMAD ABYAN ELFATHAN,L,0 Tahun - 4 Bulan - 7 Hari,3/2/2025,Ainun faziatin,MOROBAKUNG,1,1.0,5.20,61.0,0.0,-2.74,-1.63,-2.31,Gizi Kurang
3,REVA AULIA ALIENOYA,P,0 Tahun - 0 Bulan - 0 Hari,7/9/2025,SILVI RIZKI AMALIA,NGAMPEL,2,1.0,3.00,50.0,NaN,-0.52,0.46,-1.25,Gizi Baik
4,MUHAMMAD GAVIN KAIFANDRA HAFIZD,L,0 Tahun - 5 Bulan - 2 Hari,2/7/2025,nur fathihkatun nafisa,MOROBAKUNG,3,1.0,7.74,68.5,0.0,0.28,1.24,-0.53,Gizi Baik


In [3]:
gizi_df = gizi_df.drop(columns=["Nama", "Tgl Lahir", "Nama Ortu" , "Desa/Kel", "RT", "RW","ZS TB/U","ZS BB/U","ZS BB/TB"])
gizi_df.head()

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,P,1 Tahun - 5 Bulan - 0 Hari,8.80,78.0,15.0,Gizi Baik
1,L,0 Tahun - 4 Bulan - 4 Hari,6.60,62.0,0.0,Gizi Baik
2,L,0 Tahun - 4 Bulan - 7 Hari,5.20,61.0,0.0,Gizi Kurang
3,P,0 Tahun - 0 Bulan - 0 Hari,3.00,50.0,NaN,Gizi Baik
4,L,0 Tahun - 5 Bulan - 2 Hari,7.74,68.5,0.0,Gizi Baik


In [4]:
missing_value = gizi_df.mean(numeric_only=True)
gizi_df = gizi_df.fillna(missing_value)
df_imputasi = gizi_df.copy()
gizi_df.isna().sum()


JK                0
Usia Saat Ukur    0
Berat             0
Tinggi            0
LiLA              0
STATUS GIZI       0
dtype: int64

In [5]:
df_imputasi.head()

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,P,1 Tahun - 5 Bulan - 0 Hari,8.80,78.0,15.000000,Gizi Baik
1,L,0 Tahun - 4 Bulan - 4 Hari,6.60,62.0,0.000000,Gizi Baik
2,L,0 Tahun - 4 Bulan - 7 Hari,5.20,61.0,0.000000,Gizi Kurang
3,P,0 Tahun - 0 Bulan - 0 Hari,3.00,50.0,6.830678,Gizi Baik
4,L,0 Tahun - 5 Bulan - 2 Hari,7.74,68.5,0.000000,Gizi Baik


In [6]:
def conv_usia(usia_str):
    if str(usia_str).isdigit():
        return int(usia_str)
    match = re.findall(r'(\d+)', str(usia_str))

    if len(match) == 1:
        tahun = int(match[0])
        return tahun * 12

    elif len(match) >= 3:
        tahun, bulan, hari = map(int, match[:3])
        total_bulan = tahun * 12 + bulan + hari / 30
        return round(total_bulan, 1)

    elif len(match) == 2:
        tahun, bulan = map(int, match)
        total_bulan = tahun * 12 + bulan
        return int(round(total_bulan))
    return None

gizi_df["Usia Saat Ukur"] = gizi_df["Usia Saat Ukur"].apply(conv_usia)
df_conv= gizi_df.copy()
gizi_df.head(10)

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,P,17.0,8.80,78.0,15.000000,Gizi Baik
1,L,4.1,6.60,62.0,0.000000,Gizi Baik
2,L,4.2,5.20,61.0,0.000000,Gizi Kurang
3,P,0.0,3.00,50.0,6.830678,Gizi Baik
4,L,5.1,7.74,68.5,0.000000,Gizi Baik
5,L,7.4,8.80,70.9,0.000000,Gizi Baik
6,L,7.7,8.41,68.0,0.000000,Gizi Baik
7,P,8.4,7.90,70.0,0.000000,Gizi Baik
8,L,51.6,12.15,97.2,0.000000,Gizi Kurang
9,L,58.5,24.10,112.0,0.000000,Gizi Lebih


In [7]:
df_conv.head()

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,P,17.0,8.80,78.0,15.000000,Gizi Baik
1,L,4.1,6.60,62.0,0.000000,Gizi Baik
2,L,4.2,5.20,61.0,0.000000,Gizi Kurang
3,P,0.0,3.00,50.0,6.830678,Gizi Baik
4,L,5.1,7.74,68.5,0.000000,Gizi Baik


In [8]:
kolom_encode = ['JK', 'STATUS GIZI']

encoders = {}
for col in kolom_encode:
    le = LabelEncoder()
    gizi_df[col] = gizi_df[col].astype(str).str.strip()
    gizi_df[col] = le.fit_transform(gizi_df[col])
    
    encoders[col] = le
    print(f"\ hasil encoding kolom '{col}':")
    for label, encoded in zip(le.classes_, le.transform(le.classes_)):
        print(f"  {label} → {encoded}")
df_encode = gizi_df.copy()
print("\nData setelah label encoding:")
print(gizi_df.head())


\ hasil encoding kolom 'JK':
  L → 0
  P → 1
\ hasil encoding kolom 'STATUS GIZI':
  Gizi Baik → 0
  Gizi Kurang → 1
  Gizi Lebih → 2

Data setelah label encoding:
   JK  Usia Saat Ukur  Berat  Tinggi       LiLA  STATUS GIZI
0   1            17.0   8.80    78.0  15.000000            0
1   0             4.1   6.60    62.0   0.000000            0
2   0             4.2   5.20    61.0   0.000000            1
3   1             0.0   3.00    50.0   6.830678            0
4   0             5.1   7.74    68.5   0.000000            0


In [9]:
df_encode.head()

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,1,17.0,8.80,78.0,15.000000,0
1,0,4.1,6.60,62.0,0.000000,0
2,0,4.2,5.20,61.0,0.000000,1
3,1,0.0,3.00,50.0,6.830678,0
4,0,5.1,7.74,68.5,0.000000,0


In [10]:
scaler = MinMaxScaler()
kolom_numerik = gizi_df.select_dtypes(include=['int64', 'float64']).columns.drop(['STATUS GIZI', 'JK'], errors='ignore')
gizi_df[kolom_numerik] = scaler.fit_transform(gizi_df[kolom_numerik])
df_scaling = gizi_df.copy()
gizi_df.head()

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,1,0.177083,0.248175,0.452055,0.103448,0
1,0,0.042708,0.167883,0.232877,0.000000,0
2,0,0.043750,0.116788,0.219178,0.000000,1
3,1,0.000000,0.036496,0.068493,0.047108,0
4,0,0.053125,0.209489,0.321918,0.000000,0


In [11]:
df_scaling.head()

,JK,Usia Saat Ukur,Berat,Tinggi,LiLA,STATUS GIZI
0,1,0.177083,0.248175,0.452055,0.103448,0
1,0,0.042708,0.167883,0.232877,0.000000,0
2,0,0.043750,0.116788,0.219178,0.000000,1
3,1,0.000000,0.036496,0.068493,0.047108,0
4,0,0.053125,0.209489,0.321918,0.000000,0


In [12]:
# Pisahkan fitur (X) dan target (y)
X = gizi_df.drop('STATUS GIZI', axis=1)
y = gizi_df['STATUS GIZI']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Jumlah data total :", len(gizi_df))
# print("Data training     :", len(X_train))
# print("=== DATA TRAINING ===")
# print(X_train.head())
# print(y_train.head())
print(" Jumlah Data testing      :", len(X_test))
# print("\n=== DATA TESTING ===")
# print(X_test.head())

Jumlah data total : 2759
 Jumlah Data testing      : 552


DATA TRAINING

In [13]:
print("\nDistribusi kelas sebelum resampling :")
for k, v in Counter(y_train).items():
    print(f"  Kelas {k}: {v}")


Distribusi kelas sebelum resampling :
  Kelas 0: 1346
  Kelas 2: 284
  Kelas 1: 577


In [14]:
sm_gizi = SMOTE(random_state=42)
X_res, y_res = sm_gizi.fit_resample(X_train, y_train)
print('Distribusi Kelas setelah SMOTE %s' % Counter(y_res))

Distribusi Kelas setelah SMOTE Counter({0: 1346, 2: 1346, 1: 1346})


In [15]:
data = X_res.copy()
data['STATUS GIZI'] = y_res  # target kolom

fitur_numerik = ['Usia Saat Ukur', 'Berat', 'Tinggi', 'LiLA']
fitur_kategorik = ['JK']
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    ent = 0
    for i in range(len(elements)):
        p = counts[i] / np.sum(counts)
        ent -= p * math.log2(p)
    return ent
def info_gain_kategorik(data, split_attribute_name, target_name="STATUS GIZI"):
    total_entropy = entropy(data[target_name])
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)

    weighted_entropy = 0
    for i in range(len(vals)):
        subset = data[data[split_attribute_name] == vals[i]]
        weighted_entropy += (counts[i] / np.sum(counts)) * entropy(subset[target_name])

    information_gain = total_entropy - weighted_entropy
    return information_gain
def info_gain_numerik(data, split_attribute_name, target_name="STATUS GIZI"):
    mean_val = data[split_attribute_name].mean()
    total_entropy = entropy(data[target_name])

    # Bagi dua subset: < mean dan ≥ mean
    subset_kiri = data[data[split_attribute_name] < mean_val]
    subset_kanan = data[data[split_attribute_name] >= mean_val]

    # Hitung entropy terpisah
    weighted_entropy = (
        (len(subset_kiri)/len(data)) * entropy(subset_kiri[target_name]) +
        (len(subset_kanan)/len(data)) * entropy(subset_kanan[target_name])
    )

    information_gain = total_entropy - weighted_entropy
    return information_gain, mean_val
def split_info(subset_sizes, total_size):
    si = 0
    for size in subset_sizes:
        p = size / total_size
        if p > 0:
            si -= p * math.log2(p)
    return si

def gain_ratio(info_gain, subset_sizes, total_size):
    si = split_info(subset_sizes, total_size)
    if si == 0:
        return 0
    return info_gain / si
hasil_gain = []

# Fitur numerik
for col in fitur_numerik:
    ig, mean_val = info_gain_numerik(data, col, target_name="STATUS GIZI")
    subset_sizes = [
        len(data[data[col] < mean_val]),
        len(data[data[col] >= mean_val])
    ]
    gr = gain_ratio(ig, subset_sizes, len(data))
    hasil_gain.append([col, f"< {mean_val:.2f} / ≥ {mean_val:.2f}", ig, gr])

# Fitur kategorik
for col in fitur_kategorik:
    ig = info_gain_kategorik(data, col, target_name="STATUS GIZI")
    subset_sizes = data[col].value_counts().values
    gr = gain_ratio(ig, subset_sizes, len(data))
    hasil_gain.append([col, "P/L", ig, gr])

hasil_df = pd.DataFrame(hasil_gain, columns=['Fitur', 'Pembagian', 'Information Gain', 'Gain Ratio'])
hasil_df.sort_values(by='Gain Ratio', ascending=False, inplace=True)
hasil_df

,Fitur,Pembagian,Information Gain,Gain Ratio
0,Usia Saat Ukur,< 0.50 / ≥ 0.50,0.109751,0.112181
2,Tinggi,< 0.54 / ≥ 0.54,0.065665,0.065756
3,LiLA,< 0.04 / ≥ 0.04,0.031351,0.031364
1,Berat,< 0.35 / ≥ 0.35,0.030042,0.030068
4,JK,P/L,0.004347,0.004443


In [16]:
def best_split(data):
    best_feature, best_gain_ratio, best_mean, best_type = None, -1, None, None
    total_len = len(data)

    for col in fitur_numerik:
        ig, mean_val = info_gain_numerik(data, col)
        subset_sizes = [
            len(data[data[col] < mean_val]),
            len(data[data[col] >= mean_val])
        ]
        gr = gain_ratio(ig, subset_sizes, total_len)
        if gr > best_gain_ratio:
            best_feature, best_gain_ratio, best_mean, best_type = col, gr, mean_val, "numerik"

    for col in fitur_kategorik:
        ig = info_gain_kategorik(data, col)
        subset_sizes = data[col].value_counts().values
        gr = gain_ratio(ig, subset_sizes, total_len)
        if gr > best_gain_ratio:
            best_feature, best_gain_ratio, best_mean, best_type = col, gr, None, "kategorik"

    return best_feature, best_mean, best_type

def build_tree(data, max_depth=3):
    def _build_tree(subset, current_depth):
        if len(np.unique(subset['STATUS GIZI'])) == 1:
            return np.unique(subset['STATUS GIZI'])[0]

        if current_depth >= max_depth:
            return subset['STATUS GIZI'].mode()[0]

        best_feature, best_mean, best_type = best_split(subset)
        if best_feature is None:
            return subset['STATUS GIZI'].mode()[0]

        tree = {best_feature: {}}

        if best_type == "numerik":
            left_subset = subset[subset[best_feature] < best_mean]
            right_subset = subset[subset[best_feature] >= best_mean]
            tree[best_feature][f"< {best_mean:.2f}"] = _build_tree(left_subset, current_depth + 1)
            tree[best_feature][f"≥ {best_mean:.2f}"] = _build_tree(right_subset, current_depth + 1)
        else:
            for val in np.unique(subset[best_feature]):
                sub_subset = subset[subset[best_feature] == val]
                tree[best_feature][val] = _build_tree(sub_subset, current_depth + 1)
        return tree

    return _build_tree(data, 0)

def predict(sample, tree):
    for feature in tree.keys():
        value = sample[feature]
        branches = tree[feature]
        for branch, subtree in branches.items():
            branch_str = str(branch)

            if "<" in branch_str:
                threshold = float(branch_str.split()[1])
                if value < threshold:
                    return predict(sample, subtree) if isinstance(subtree, dict) else subtree
            elif "≥" in branch_str:
                threshold = float(branch_str.split()[1])
                if value >= threshold:
                    return predict(sample, subtree) if isinstance(subtree, dict) else subtree
            else:
                if str(value) == branch_str:
                    return predict(sample, subtree) if isinstance(subtree, dict) else subtree
    return None
depth=5
tree_c50 = build_tree(data, max_depth=depth)
# print("\n=== Struktur Pohon C5.0 ===")
# print(tree_c50)

y_pred = [predict(data.iloc[i], tree_c50) for i in range(len(data))]
y_true = data['STATUS GIZI'].values
akurasi = np.mean(np.array(y_pred) == np.array(y_true))
print(f"\nAkurasi model algoritma C5.0 Tunggal dengn kedalaman pohon {depth} : {akurasi:.4f}")


Akurasi model algoritma C5.0 Tunggal dengn kedalaman pohon 5 : 0.8021


In [17]:
def adaboost_M1_train(data, n_estimators, max_depth, target_col="STATUS GIZI"):
    n = len(data)
    classes = np.unique(data[target_col])
    K = len(classes)

    # Pisahkan fitur dan target
    X = data.drop(columns=[target_col])
    y = data[target_col].values

    # Bobot awal
    w = np.ones(n) / n  

    models = []
    alphas = []
    errors = []

    majority_class = data[target_col].mode()[0]

    for t in range(n_estimators):

        # Sampling berbobot
        idx_sample = np.random.choice(n, size=n, replace=True, p=w)
        X_sample = X.iloc[idx_sample].copy()
        y_sample = y[idx_sample]

        # Gabungkan agar build_tree mengenali target
        sample = X_sample.copy()
        sample[target_col] = y_sample

        # Train weak learner (C5.0 versi Anda)
        model = build_tree(sample, max_depth=max_depth)

        # Prediksi seluruh data training
        y_pred = []
        for _, row in X.iterrows():
            pred = predict(row, model)

            # Pastikan prediksi valid
            if pred not in classes:
                pred = majority_class

            y_pred.append(pred)

        y_pred = np.array(y_pred)

        # Hitung error berbobot
        misclassified = (y_pred != y)
        error = np.sum(w * misclassified)

        # if error >= 0.5:
        #     print(f"Iterasi {t+1} dihentikan karena error >= 0.5 ({error:.4f})")
        #     break

        # Rumus alpha SAMME (multi-class)
        error = max(error, 1e-10)
        alpha = np.log((1 - error) / error) + np.log(K - 1)

        # Simpan
        models.append(model)
        alphas.append(alpha)
        errors.append(error)

        # Update bobot
        w *= np.exp(alpha * misclassified)
        w /= np.sum(w)

        # print(f"Iterasi {t+1}: error={error:.4f}, alpha={alpha:.4f}")

    return models, alphas, classes


def adaboost_M1_predict(data, models, alphas, classes):
    X = data.copy()
    n = len(X)
    final_pred = []

    for i in range(n):
        votes = {c: 0 for c in classes}

        for alpha_t, model in zip(alphas, models):
            pred = predict(X.iloc[i], model)
            if pred in votes:
                votes[pred] += alpha_t

        final_pred.append(max(votes, key=votes.get))

    return np.array(final_pred)

# ==========================================================
# 🔹 LATIH DAN UJI MODEL
# ==========================================================
estimator = 100
depth = 5
models, alphas, classes = adaboost_M1_train(data, n_estimators=estimator, max_depth=depth, target_col="STATUS GIZI")
y_true = data["STATUS GIZI"].values
y_pred = adaboost_M1_predict(data, models, alphas, classes)

akurasi = np.mean(y_true == y_pred)
print(f"\n🎯 Akurasi AdaBoost (C5.0) pada data training: {akurasi:.4f}")


🎯 Akurasi AdaBoost (C5.0) pada data training: 0.8987


In [18]:
cm_training = confusion_matrix(y_true, y_pred, labels=classes)
print("\n=== Confusion Matrix ===")
print(cm_training)
print("\n=== Precision, Recall, F1-Score per Kelas ===")
print(classification_report(y_true, y_pred, target_names=[str(c) for c in classes]))
print("\n=== Specificity dan G-Mean per Kelas ===")

for idx, cls in enumerate(classes):
    TP = cm_training[idx, idx]
    FN = cm_training[idx, :].sum() - TP
    FP = cm_training[:, idx].sum() - TP
    TN = cm_training.sum() - (TP + FN + FP)

    # Hindari pembagian nol
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0   # recall
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    gmean = np.sqrt(sensitivity * specificity)

    print(f"\nKelas: {cls}")
    print(f"  TP = {TP}")
    print(f"  FP = {FP}")
    print(f"  FN = {FN}")
    print(f"  TN = {TN}")
    print(f"  Recall/Sensitivity = {sensitivity:.4f}")
    print(f"  Specificity        = {specificity:.4f}")
    print(f"  G-Mean             = {gmean:.4f}")


=== Confusion Matrix ===
[[1083  169   94]
 [  70 1276    0]
 [  76    0 1270]]

=== Precision, Recall, F1-Score per Kelas ===
              precision    recall  f1-score   support

           0       0.88      0.80      0.84      1346
           1       0.88      0.95      0.91      1346
           2       0.93      0.94      0.94      1346

    accuracy                           0.90      4038
   macro avg       0.90      0.90      0.90      4038
weighted avg       0.90      0.90      0.90      4038


=== Specificity dan G-Mean per Kelas ===

Kelas: 0
  TP = 1083
  FP = 146
  FN = 263
  TN = 2546
  Recall/Sensitivity = 0.8046
  Specificity        = 0.9458
  G-Mean             = 0.8723

Kelas: 1
  TP = 1276
  FP = 169
  FN = 70
  TN = 2523
  Recall/Sensitivity = 0.9480
  Specificity        = 0.9372
  G-Mean             = 0.9426

Kelas: 2
  TP = 1270
  FP = 94
  FN = 76
  TN = 2598
  Recall/Sensitivity = 0.9435
  Specificity        = 0.9651
  G-Mean             = 0.9542


DATA TESTING

In [20]:
data_test = X_test.copy()
data_test["STATUS GIZI"] = y_test

y_true_test = data_test["STATUS GIZI"].values
y_pred_test = [predict(data_test.iloc[i], tree_c50) for i in range(len(data_test))]

akurasi_test = np.mean(np.array(y_pred_test) == y_true_test)
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np

# Konversi ke numpy array
y_pred_test = np.array(y_pred_test)
y_true_test = np.array(y_true_test)

print("\n=== Evaluasi pada DATA TESTING ===")

# Confusion Matrix
labels_test = np.unique(y_true_test)
cm_test = confusion_matrix(y_true_test, y_pred_test, labels=labels_test)
print("\nConfusion Matrix (Testing)")
print("Labels:", labels_test)
print(cm_test)

# Precision, Recall (Sensitivity), dan F1-Score
precision_test = precision_score(y_true_test, y_pred_test, labels=labels_test, average=None, zero_division=0)
recall_test = recall_score(y_true_test, y_pred_test, labels=labels_test, average=None, zero_division=0)
f1_test = f1_score(y_true_test, y_pred_test, labels=labels_test, average=None, zero_division=0)

# G-Mean per kelas
print("\n=== Metode Evaluasi Per Kelas (Testing) ===")
for i, label in enumerate(labels_test):
    TP = cm_test[i, i]
    FN = cm_test[i, :].sum() - TP
    FP = cm_test[:, i].sum() - TP
    TN = cm_test.sum() - (TP + FN + FP)

    sensitivity = recall_test[i]
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    g_mean = np.sqrt(sensitivity * specificity)

    print(f"\nKelas: {label}")
    print(f"Precision   : {precision_test[i]:.4f}")
    print(f"Recall      : {recall_test[i]:.4f} (Sensitivity)")
    print(f"F1-Score    : {f1_test[i]:.4f}")
    print(f"G-Mean      : {g_mean:.4f}")

print(f"Akurasi C5.0 pada DATA TESTING (depth={depth}): {akurasi_test:.4f}")


=== Evaluasi pada DATA TESTING ===

Confusion Matrix (Testing)
Labels: [0 1 2]
[[229  63  45]
 [ 18 120   6]
 [ 13   9  49]]

=== Metode Evaluasi Per Kelas (Testing) ===

Kelas: 0
Precision   : 0.8808
Recall      : 0.6795 (Sensitivity)
F1-Score    : 0.7672
G-Mean      : 0.7626

Kelas: 1
Precision   : 0.6250
Recall      : 0.8333 (Sensitivity)
F1-Score    : 0.7143
G-Mean      : 0.8284

Kelas: 2
Precision   : 0.4900
Recall      : 0.6901 (Sensitivity)
F1-Score    : 0.5731
G-Mean      : 0.7855
Akurasi C5.0 pada DATA TESTING (depth=5): 0.7210


In [ ]:
data_test = X_test.copy()      # hanya fitur
y_truetest = y_test           # target asli

y_predtest = adaboost_M1_predict(
    data_test,
    models,
    alphas,
    classes
)

akurasi_test = np.mean(y_truetest == y_predtest)
print("Akurasi testing:", akurasi_test)

Akurasi testing: 0.8496376811594203


In [ ]:
cm_test = confusion_matrix(y_truetest, y_predtest, labels=classes)

print("\n=== Confusion Matrix (Testing) ===")
print(cm_test)

print("\n=== Precision, Recall, F1-Score per Kelas ===")
print(classification_report(
    y_truetest,
    y_predtest,
    labels=classes,
    target_names=[str(c) for c in classes]
))

print("\n=== Specificity dan G-Mean per Kelas ===")

gmeans = []   # Menyimpan g-mean tiap kelas

for idx, cls in enumerate(classes):

    TP = cm_test[idx, idx]
    FN = cm_test[idx, :].sum() - TP
    FP = cm_test[:, idx].sum() - TP
    TN = cm_test.sum() - (TP + FN + FP)

    # Hindari pembagian nol
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    gmean = np.sqrt(recall * specificity)

    gmeans.append(gmean)

    print(f"\nKelas: {cls}")
    print(f"  TP = {TP}")
    print(f"  FP = {FP}")
    print(f"  FN = {FN}")
    print(f"  TN = {TN}")
    print(f"  Recall/Sensitivity = {recall:.4f}")
    print(f"  Specificity        = {specificity:.4f}")
    print(f"  G-Mean             = {gmean:.4f}")


gmean_macro = np.mean(gmeans)

print(f"\n=== G-Mean Total (Macro) = {gmean_macro:.4f}")



=== Confusion Matrix (Testing) ===
[[280  32  25]
 [ 12 129   3]
 [ 10   1  60]]

=== Precision, Recall, F1-Score per Kelas ===
              precision    recall  f1-score   support

           0       0.93      0.83      0.88       337
           1       0.80      0.90      0.84       144
           2       0.68      0.85      0.75        71

    accuracy                           0.85       552
   macro avg       0.80      0.86      0.82       552
weighted avg       0.86      0.85      0.85       552


=== Specificity dan G-Mean per Kelas ===

Kelas: 0
  TP = 280
  FP = 22
  FN = 57
  TN = 193
  Recall/Sensitivity = 0.8309
  Specificity        = 0.8977
  G-Mean             = 0.8636

Kelas: 1
  TP = 129
  FP = 33
  FN = 15
  TN = 375
  Recall/Sensitivity = 0.8958
  Specificity        = 0.9191
  G-Mean             = 0.9074

Kelas: 2
  TP = 60
  FP = 28
  FN = 11
  TN = 453
  Recall/Sensitivity = 0.8451
  Specificity        = 0.9418
  G-Mean             = 0.8921

=== G-Mean Total (Macr

In [ ]:
import pickle

In [ ]:
pickle.dump({      # dataframe hasil imputasi
    'dok_misval': df_imputasi,
    'dok_encoded': df_encode,
    'dok_scaled': df_scaling,
    'dok_usia' : df_conv,
    'split': {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test
    },
    'input_features': X_train.columns.tolist(),
    'smote': {
        'X_res': X_res,
        'y_res': y_res
    },
    'missing_value': missing_value,
    'encoder': encoders,
    'scaler': scaler,
     'algoritma.C5': {
        'tree': tree_c50,
        'depth': depth
    },
    'adaboost': {
        'models': models,
        'alphas': alphas,
        'classes': classes
    },
    'X_test': X_test,             
    'y_test': y_test,
    'X_smote': X_res,             
    'y_smote': y_res,
}, open('model_gizi.sav', 'wb'))
